In [1]:
import numpy as np
import torch
import itertools
from datetime import datetime
import sys
import pandas as pd

In [2]:
df = pd.read_csv("sunspot_data.csv")
print("DataFrame Shape: {} rows, {} columns".format(*df.shape))
display(df.head())

DataFrame Shape: 73718 rows, 9 columns


,Unnamed: 0,Year,Month,Day,Date In Fraction Of Year,Number of Sunspots,Standard Deviation,Observations,Indicator
0,0,1818,1,1,1818.001,-1,-1.0,0,1
1,1,1818,1,2,1818.004,-1,-1.0,0,1
2,2,1818,1,3,1818.007,-1,-1.0,0,1
3,3,1818,1,4,1818.010,-1,-1.0,0,1
4,4,1818,1,5,1818.012,-1,-1.0,0,1


In [3]:
uni_data = df['Number of Sunspots']
uni_data.index = df['Date In Fraction Of Year']
uni_data.head()

Date In Fraction Of Year
1818.001   -1
1818.004   -1
1818.007   -1
1818.010   -1
1818.012   -1
Name: Number of Sunspots, dtype: int64

In [4]:
TRAIN_SPLIT = 8000
uni_data = uni_data.values[:10000]
#uni_data = uni_data.astype(np.float64) # convert to float for matrix multiplication later in training
'''why need to normalize/standardize -- it only makes for multiple features (we need to standarize them)'''
uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
uni_train_std = uni_data[:TRAIN_SPLIT].std()
uni_data = (uni_data-uni_train_mean)/uni_train_std

In [5]:
# Data loader to create context window
def univariate_data(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
        
    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        #Reshape data from (history_size,) to (history_size, 1) to make each input x an array
        data.append(np.reshape(dataset[indices], (history_size, 1, 1)))
        #data.append(dataset[indices])
        targetIndices = range(i, i+target_size)
        labels.append(np.reshape(dataset[targetIndices], (target_size, 1, 1)))
        #labels.append(dataset[targetIndices])
    return np.array(data), np.array(labels)

In [6]:
univariate_past_history = 100
#n-1
univariate_future_target = 30

x_train_uni, y_train_uni = univariate_data(dataset=uni_data,
                                           start_index=0,
                                           end_index=TRAIN_SPLIT,
                                           history_size=univariate_past_history,
                                           target_size=univariate_future_target)
x_val_uni, y_val_uni = univariate_data(dataset=uni_data,
                                       start_index=TRAIN_SPLIT,
                                       end_index=None,
                                       history_size=univariate_past_history,
                                       target_size=univariate_future_target)

In [7]:
print ('Single window of past history. Shape: {}'.format(x_train_uni[0].shape))
#print (x_train_uni[0])
# print ('\n Target temperature to predict. Shape: {}'.format(y_train_uni[0].shape))
# print (y_train_uni[0])
#print(len(x_train_uni[0][0]))

Single window of past history. Shape: (100, 1, 1)


In [8]:
x_train_uni.shape

(7900, 100, 1, 1)

In [9]:
x_val_uni.shape

(1870, 100, 1, 1)

In [10]:
#x_train_uni[:100]

In [11]:
y_train_uni.shape

(7900, 30, 1, 1)

In [12]:
y_val_uni.shape

(1870, 30, 1, 1)

In [13]:
class Tanh:
    def forward(self, x):
        return torch.tanh(x)

    def backwardWithTanhValue(self, tanh, top_diff):
        ## at this activation function layer, we should use * --- which is element-wise multiplication
        return (1.0 - torch.square(tanh)) * top_diff

In [14]:
class MultiplyGate:
    def forward(self, W, x):
        return torch.matmul(W, x)
    def backward(self, W, s, dy_pred):
    #dy_pred is [f,1]
        dW = torch.matmul(dy_pred, torch.transpose(s, 0, 1))
        ds = torch.matmul(torch.transpose(W, 0, 1), dy_pred)
        return dW, ds

class AddGate:
    def forward(self, x1, x2):
        return x1 + x2
    ''' no need of backward func at all since the gradient of addition is 1'''


class OutputGate:
    def predict(self, mulv, b):
        return mulv + b

    def lossValue(self, y_pred, y):
        #if abs(y - y_pred) > 1000:
          # print("y")
          # print(y)
          # print("y_pred")
          # print(y_pred)
        return abs(y - y_pred)
    
    # -sign(y - y_pred) -- note: the order of y and y_pred doesn't matter once you keep in consistent
    # this is derative of vector respect with a vector in same shap -- hence, it is element-wise
    # ∂(a.T@x)/∂x = a where x is vector
    def backward(self, y_pred, y):
        return -torch.sign(y - y_pred)

In [15]:
mulGate = MultiplyGate()
addGate = AddGate()
activation = Tanh()

class RNNLayer:
    #x is input -- word vector; s is hidden state vector, U, W, V is matrix
    def forward(self, x, prev_s, U, W, V, b_h):
        self.input = x
        self.mulu = mulGate.forward(U, x)
        self.mulw = mulGate.forward(W, prev_s)
        self.add = addGate.forward(self.mulw, self.mulu)
        self.add = addGate.forward(self.add, b_h)
        self.s = activation.forward(self.add)
        self.mulv = mulGate.forward(V, self.s)

    #all parameters are tensor
    def backward(self, prev_s, U, W, V, diff_s, dy_pred):
        d_by = dy_pred
        dV, dy_predV = mulGate.backward(V, self.s, dy_pred)
        # diff_s is not always a vector of 0 --- for back trancate, it is not. value gets acculumated
        ds = dy_predV + diff_s
        #optimization: replace self.add with self.s directly
        dadd = activation.backwardWithTanhValue(self.s, ds)
        d_bh = dadd
        # the usage of dprev_s???? -- used as ds in the back truncate (its value will be assigned to diff_s and the new dsv will be 0 since dmulv is 0)
        dW, dprev_s = mulGate.backward(W, prev_s, dadd)
        # calculating dx is a waste  
        dU, dx = mulGate.backward(U, self.input, dadd)
      
        return (dprev_s, dU, dW, dV, d_by, d_bh)

In [16]:
class Model:
    def __init__(self, f, feature_dim, hidden_dim=100, bptt_truncate=4):
        self.feature_dim = feature_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        self.f = f
        
        self.dtype = torch.float64
        self.device = torch.device("cuda:0")
        self.tensor = torch.tensor((), dtype=self.dtype, device = self.device)

        self.U = torch.from_numpy(np.random.uniform(-np.sqrt(1. / feature_dim), np.sqrt(1. / feature_dim), (hidden_dim, feature_dim))).to(self.device)
        self.W = torch.from_numpy(np.random.uniform(-np.sqrt(1. / hidden_dim), np.sqrt(1. / hidden_dim), (hidden_dim, hidden_dim))).to(self.device)
        self.V = torch.from_numpy(np.random.uniform(-np.sqrt(1. / hidden_dim), np.sqrt(1. / hidden_dim), (feature_dim, hidden_dim))).to(self.device)
        self.b_h = torch.from_numpy(np.random.uniform(-np.sqrt(1. / hidden_dim), np.sqrt(1. / hidden_dim), (hidden_dim, 1))).to(self.device)
        self.b_y = torch.from_numpy(np.random.uniform(-np.sqrt(1. / feature_dim), np.sqrt(1. / feature_dim), (feature_dim, 1))).to(self.device)
      

        #initial orthogonal matrices
        u, s, vh = torch.linalg.svd(self.W, full_matrices=False)
        self.W = u @ vh
        #full_matrices(bool, optional) : If True (default), u and vh have the shapes (…, M, M) and (…, N, N), respectively. 
        #Otherwise, the shapes are (…, M, K) and (…, K, N), respectively, where K = min(M, N).

    # x is (history, features) while y is (targets, features)
    def forward_propagation(self, x, y, teacherForcing):
        #when there is no prev state, we use tensor of 0
        prev_s = self.tensor.new_zeros(self.hidden_dim, 1)
        output = OutputGate()
        last_predict_y = self.tensor.new_zeros(self.feature_dim, 1)
        layers = []
        # For each time step in the history window
        for t in range(len(x)):
            layer = RNNLayer()
            #input represents all features in that time stamp -- (features, 1)
            input = x[t]
            layer.forward(input, prev_s, self.U, self.W, self.V, self.b_h)
            layers.append(layer)
            prev_s = layer.s
            if (t == (len(x) - 1)):
              last_predict_y = output.predict(layer.mulv, self.b_y)
        
        # auto-regressive
        # one layer for each time step in target window
        yPreds = []
        loss = 0.0
        # For each time step in the target window
        for t in range(len(y)):
            yPreds.append(last_predict_y)
            loss += output.lossValue(last_predict_y, y[t])

            layer = RNNLayer()
            if teacherForcing:
              '''teacher forcing:'''
              input = y[t]
            else:
              '''original rnn:'''
              input = last_predict_y
            
            layer.forward(input, prev_s, self.U, self.W, self.V, self.b_h)
            last_predict_y = output.predict(layer.mulv, self.b_y)

            layers.append(layer)
            prev_s = layer.s
        
        lss = loss / float(len(y))
        return lss, layers, yPreds, len(x)

    def bptt(self, y, layers, yPreds, historyLen):
        output = OutputGate()

        dU = self.tensor.new_zeros(self.U.shape)
        dV = self.tensor.new_zeros(self.V.shape)
        dW = self.tensor.new_zeros(self.W.shape)
        db_h = self.tensor.new_zeros(self.b_h.shape)
        db_y = self.tensor.new_zeros(self.b_y.shape)

        diff_s = self.tensor.new_zeros(self.hidden_dim, 1)
        #bptt -- timestampes -- future prediction
        for t in range(len(y)):
            #d|y - y^|/dy^ -- f x 1 shape -- 1/m * -sign(y - y_pred)
            dy_pred = output.backward(yPreds[t], y[t])/len(y)
            prev_s_t = layers[t+historyLen-1].s
            #shape of input is [1,1]
            dprev_s, dU_t, dW_t, dV_t, d_by_t, d_bh_t = layers[t].backward(prev_s_t, self.U, self.W, self.V, diff_s, dy_pred)

            # the reson of using this inner loop? -- sum up the impacts of s_t, s_t-1, s_t-2, s_t-3, S_t-4 on loss functions
            # the reason of initializing dy be 0 vector in shape of f x 1 --- to make ds be 0 and thus make ds == dprev_s (since ds = ds + diff)
            dy_hat = self.tensor.new_zeros(self.feature_dim, 1)
            # back track last historyLen - 1 layers
            for i in range(t + historyLen - 1, 0, -1):
                prev_s_i = layers[i-1].s
                dprev_s, dU_i, dW_i, dV_i, d_by_i, d_bh_i = layers[i].backward(prev_s_i, self.U, self.W, self.V, dprev_s, dy_hat)
                #sum up the impacts of s_t, s_t-1, s_t-2, s_t-3, S_t-4 on loss functions
                dU_t += dU_i
                dW_t += dW_i
                d_bh_t += d_bh_i
                
            dV += dV_t
            db_y += d_by_t
            dU += dU_t
            dW += dW_t
            db_h += d_bh_t
            

        return (dU, dW, dV, db_h, db_y)

    def sgd_step(self, x, y, learning_rate):
        #The dtype for an array of arrays will always be object. This is unavoidable because with NumPy only non-jagged n-dimensional arrays can be held in a contiguous memory block.
        x = torch.tensor(x, dtype=self.dtype, device=self.device)
        y = torch.tensor(y, dtype=self.dtype, device=self.device)
        #x is a sentence (aka one example) composed of many words
        lss, layers, yPreds, historyLen = self.forward_propagation(x, y, True)
        dU, dW, dV, db_h, db_y = self.bptt(y, layers, yPreds, historyLen)
        self.U -= learning_rate * dU
        self.V -= learning_rate * dV
        self.b_h -= learning_rate * db_h
        self.b_y -= learning_rate * db_y
        self.W -= learning_rate * dW
        #don't do orthogonize W at every sgd
        #u, s, vh = torch.linalg.svd(self.W, full_matrices=False)
        #self.W = u @ vh
        
        return lss
        #This is just for testing in this code
        # print("W's learning rate is %f"%(np.linalg.norm(learning_rate * dW)))
        # eigvals = np.linalg.eigvals(self.W)
        # largestValueIndex = np.argmax(eigvals)
        # print("W's largest eigenValue is %f"%(eigvals[largestValueIndex]))

    def train(self, X, Y, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
        num_examples_seen = 0
        losses = []
        for epoch in range(nepoch):
            loss = 0

            # For each training example...
            for i in range(len(Y)):
                '''sgd return the loss and accureacy of this sentence/example'''
                #print(i)
                lss = self.sgd_step(X[i], Y[i], learning_rate)
                # print("loss of a sample")
                # print(lss)
                loss += lss
                num_examples_seen += 1

            if (epoch % evaluate_loss_after == 0):
                #1 place
                #loss, accuracy = self.calculate_total_loss_and_predict_accuracy(X, Y)
                loss /= len(Y)
                losses.append(str(float(loss)))
                time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print("%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss))
                f.write("%s: Loss after num_examples_seen=%d epoch=%d: %f\n" % (time, num_examples_seen, epoch, loss))
                f.flush()
                # optional: Adjust the learning rate if loss increases
                # if len(losses) > 1 and losses[-1][1] > losses[-2][1]:
                #     learning_rate = learning_rate * 0.5
                #     print("Setting learning rate to %f" % learning_rate)
                sys.stdout.flush()
        return losses

    def justForwardPropagation(self, x, y):
        x = torch.tensor(x, dtype=self.dtype, device=self.device)
        y = torch.tensor(y, dtype=self.dtype, device=self.device)
        lss, _, _, _ = self.forward_propagation(x, y, False)
        return lss

    def calculate_testing_loss(self, X, Y):
        # For each testing example...
        loss = 0
        for i in range(len(Y)):
            lss = self.justForwardPropagation(X[i], Y[i])
            loss += lss
        loss /= len(Y)
        print("Testing loss value=%f" % (loss))
        f.write("Testing loss value=%f\n" % (loss))
        f.flush()



In [17]:
feature_dim = 1
hidden_dim = 100

In [18]:
random_seed = 10 # or any of your favorite number 
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
f = open("RNN_origin_nonOrth_timeSeries_backThrough_loss_log.txt", "w")
rnn = Model(f, feature_dim, hidden_dim, univariate_past_history)
losses = rnn.train(x_train_uni, y_train_uni, learning_rate=0.0001, nepoch=200, evaluate_loss_after=1)

In [ ]:
with open("RNN_origin_nonOrth_timeSeries_backThrough_loss_array.txt", "w") as txt_file:
  txt_file.write(", ".join(losses))

In [ ]:
rnn.calculate_testing_loss(x_val_uni, y_val_uni)
# print("Test Data Prediction accuracy=%f" % (accuracy))
# f.write("Test Data Prediction accuracy=" + str(accuracy))
f.close()

In [ ]:
# Convert the PyTorch tensor to a NumPy array
numpy_matrix = rnn.U.cpu().numpy()
# Save the NumPy array to a CSV file
file_path = 'RNN_origin_nonOrth_timeSeries_backThrough_U.csv'
np.savetxt(file_path, numpy_matrix, delimiter=',')

numpy_matrix = rnn.W.cpu().numpy()
file_path = 'RNN_origin_nonOrth_timeSeries_backThrough_W.csv'
np.savetxt(file_path, numpy_matrix, delimiter=',')

numpy_matrix = rnn.V.cpu().numpy()
file_path = 'RNN_origin_nonOrth_timeSeries_backThrough_V.csv'
np.savetxt(file_path, numpy_matrix, delimiter=',')

numpy_matrix = rnn.b_y.cpu().numpy()
file_path = 'RNN_origin_nonOrth_timeSeries_backThrough_by.csv'
np.savetxt(file_path, numpy_matrix, delimiter=',')

numpy_matrix = rnn.b_h.cpu().numpy()
file_path = 'RNN_origin_nonOrth_timeSeries_backThrough_bh.csv'
np.savetxt(file_path, numpy_matrix, delimiter=',')

https://songhuiming.github.io/pages/2017/08/20/build-recurrent-neural-network-from-scratch/

has the math proof of bptt_truncate

https://towardsdatascience.com/implementing-recurrent-neural-network-using-numpy-c359a0a68a67

https://mp.weixin.qq.com/s?__biz=MzIzODExMDE5MA==&mid=2694182661&idx=1&sn=ddfb3f301f5021571992824b21ddcafe&chksm=cc5f0384fb288a92877fbee9c6a1a03ba68e375b1552e762567d71d70105f67aacec7def8c40#rd

https://github.com/nicklashansen/rnn_lstm_from_scratch


differen paradigm -- has optimization
